In [2]:
import geopandas as gpd
import pandas as pd
import foursquare
from yelpapi import YelpAPI

from shapely.geometry import Point

## Yelp

In [3]:
yelp_api = YelpAPI('dEW7W1vrAwOL1XDtf3dwTA',
                   'hFskWujgXbtGYX0Q3snLngJWk19svrY9FWNTKBRaPGxs41WFAg96NGeHFnPCFuxz')

In [61]:
def yelp_3rdwave(offset=0):
    return yelp_api.search_query(
        location="New York, NY", 
        term="third wave coffee", 
        categories=['coffee'],
        limit=50,
        offset=offset
    )

In [69]:
yelp_3rdwave()['businesses'][0]

{'categories': [{'alias': 'coffee', 'title': 'Coffee & Tea'}],
 'coordinates': {'latitude': 40.7456663381275, 'longitude': -73.9880916348562},
 'display_phone': '(855) 711-3385',
 'distance': 4502.935412142,
 'id': 'stumptown-coffee-roasters-new-york',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/ShLJSEuevRDUKfb35loEXg/o.jpg',
 'is_closed': False,
 'location': {'address1': 'The Ace Hotel',
  'address2': '20 W 29th St',
  'address3': '',
  'city': 'New York',
  'country': 'US',
  'display_address': ['The Ace Hotel', '20 W 29th St', 'New York, NY 10001'],
  'state': 'NY',
  'zip_code': '10001'},
 'name': 'Stumptown Coffee Roasters',
 'phone': '+18557113385',
 'price': '$$',
 'rating': 4.5,
 'review_count': 1222,
 'transactions': [],
 'url': 'https://www.yelp.com/biz/stumptown-coffee-roasters-new-york?adjust_creative=dEW7W1vrAwOL1XDtf3dwTA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=dEW7W1vrAwOL1XDtf3dwTA'}

In [70]:
name = []
geometry = []
address = []
ids = []
ratings = []

offset = 0
total_results = yelp_3rdwave()['total']

print('Total Coffeeshops:', total_results)

while offset < total_results:
    print(offset)    
    for biz in yelp_3rdwave(offset)['businesses']:        
        name.append(biz['name'])
        geometry.append(Point(biz['coordinates']['longitude'], biz['coordinates']['latitude']))
        address.append(" ".join(biz['location']['display_address']))
        ids.append(biz['id'])
        ratings.append(biz['rating'])
    
    offset += 50

Total Coffeeshops: 375
0
50
100
150
200
250
300
350


In [71]:
columns = {'coffeeshop_name': name, 'geometry': geometry, 'address': address, 'yelp_id': ids}
yelpDf = gpd.GeoDataFrame(columns)

In [72]:
len(yelpDf)

375

In [89]:
yelpDf.to_file('Data/coffee.geojson')

## Yelp Beer

In [81]:
def yelp_beer(offset=0):
    return yelp_api.search_query(
        location="New York, NY", 
        term="brewery", 
        categories=['breweries'],
        limit=50,
        offset=offset
    )

In [84]:
name = []
geometry = []
address = []
ids = []
ratings = []

offset = 0
total_results = yelp_beer()['total']

print('Total Breweries:', total_results)

while offset < total_results:
    print(offset)    
    for biz in yelp_beer(offset)['businesses']:        
        name.append(biz['name'])
        geometry.append(Point(biz['coordinates']['longitude'], biz['coordinates']['latitude']))
        address.append(" ".join(biz['location']['display_address']))
        ids.append(biz['id'])
        ratings.append(biz['rating'])
    
    offset += 50

Total Breweries: 67
0
50


In [85]:
columns = {'brewery_name': name, 'geometry': geometry, 'address': address, 'yelp_id': ids}
yelpBeer = gpd.GeoDataFrame(columns)

In [88]:
yelpBeer.to_file('Data/yelp_beer.geojson')

## Foursquare

In [3]:
client = foursquare.Foursquare(client_id='1ZZN5DWC55SKPPS5APYUIP41GEQET2U03UPL1J4M2N1XXBWM',
                               client_secret='J5SBGX1O2TRIFTR4VDM4CTMTST03QEFXU0S4AODDBWDSPEYM')

In [50]:
roasters = client.venues.search(params={
                                        'near': 'New York, NY',
                                        'categoryId': '4bf58dd8d48988d1e0931735',
                                        'limit': 50})

In [52]:
roasters['venues']

50

In [53]:
name = []
geometry = []
address = []
ids = []

for v in roasters['venues']:
    name.append(v['name'])
    geometry.append(Point(v['location']['lng'], v['location']['lat']))
    address.append(" ".join(v['location']['formattedAddress']))
    ids.append(v['id'])

In [54]:
columns = {'coffeeshop_name': name, 'geometry': geometry, 'address': address, 'foursquare_id': ids}
coffee = gpd.GeoDataFrame(columns)

In [67]:
coffee.to_csv('Data/coffee.csv')

In [56]:
# Use list of census tract centroids to find a more comprehensive list of coffeeshops 